# Backend : iteraction with user

In [1]:
# import
import os
import folium
import pyproj
from functools import partial
import pandas as pd
from shapely.ops import transform
import geopandas as gpd
import openrouteservice as ors
from IPython.display import display
from shapely.geometry import Point, LineString, Polygon

/Users/mac/Desktop/2R/env/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### User input coordinates : addresses later.

In [2]:
# input coordinates
coordinates = [[2.3668617010116577,48.86100925394748 ], [2.4125343561172485, 48.83392954811057]]

In [53]:
# openrouteservices 
client = ors.Client(key='5b3ce3597851110001cf62485169f9e2d05c419eaf0607fe2b5b0bfa')

#requete openrouteservice
route = client.directions(
    coordinates=coordinates,
    profile='cycling-regular',
    format ='geojson',
    units ='km'
)

In [60]:
# get clusters
cluster_polygons_file = "../model/model_cluster/cluster_polygons.geojson"
gdf_cluster = gpd.read_file(cluster_polygons_file)
display(gdf_cluster.head(2))

# clusters with buffer
orig_crs = 4326
fr_crs = 2154
gdf_cluster = gdf_cluster.to_crs(epsg=fr_crs)

# crs
buffer_m = 15
gdf_cluster['geometry'] = gdf_cluster.geometry.buffer(buffer_m)
#gdf_cluster = gdf_cluster.buffer(buffer_m)

# set crs
gdf_cluster = gdf_cluster.to_crs(epsg=orig_crs)

type(gdf_cluster)
gdf_cluster

,cluster_index,geometry
0,0,"POLYGON ((2.41149 48.84703, 2.41106 48.84705, ..."
1,1,"POLYGON ((2.36897 48.85400, 2.36893 48.85403, ..."


,cluster_index,geometry
0,0,"POLYGON ((2.41169 48.84707, 2.41170 48.84705, ..."
1,1,"POLYGON ((2.36917 48.85403, 2.36917 48.85401, ..."
2,2,"POLYGON ((2.37251 48.84846, 2.37249 48.84846, ..."
3,3,"POLYGON ((2.31386 48.86467, 2.31386 48.86465, ..."
4,4,"POLYGON ((2.29898 48.86840, 2.29897 48.86839, ..."
...,...,...
139,139,"POLYGON ((2.37868 48.85880, 2.37869 48.85879, ..."
140,140,"POLYGON ((2.36493 48.85404, 2.36495 48.85404, ..."
141,141,"POLYGON ((2.35992 48.87225, 2.35993 48.87223, ..."
142,142,"POLYGON ((2.37064 48.89819, 2.37062 48.89819, ..."


In [61]:
# user route to df
user_line_df = pd.DataFrame(route['features'][0]['geometry']['coordinates'],
                            columns=["lon", "lat"])

# df to gdf
user_line_gdf = gpd.GeoDataFrame(user_line_df, 
                    geometry=gpd.points_from_xy(user_line_df.lon, user_line_df.lat),
                    crs="epsg:4326")

display(user_line_gdf.sample(2))

,lon,lat,geometry
30,2.372631,48.862187,POINT (2.37263 48.86219)
22,2.370040,48.863729,POINT (2.37004 48.86373)


In [62]:
# intersect : spatial join
cluster_on_route = gpd.sjoin(gdf_cluster, user_line_gdf)[gdf_cluster.columns]

In [63]:
cluster_on_route

,cluster_index,geometry
18,18,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."
18,18,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."
18,18,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."
18,18,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."
18,18,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."
36,36,"POLYGON ((2.37022 48.86366, 2.37021 48.86365, ..."
36,36,"POLYGON ((2.37022 48.86366, 2.37021 48.86365, ..."
68,68,"POLYGON ((2.37695 48.85987, 2.37697 48.85986, ..."
91,91,"POLYGON ((2.38569 48.85466, 2.38570 48.85465, ..."
91,91,"POLYGON ((2.38569 48.85466, 2.38570 48.85465, ..."


In [77]:
# localisation Paris
m = folium.Map(location=[48.862, 2.346], tiles='cartodbpositron', zoom_start=12)

# cluster on map
gdf = gdf_cluster
for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: 
                           {'fillColor': '#2a9d8f',
                            'color': '#2a9d8f'}
    )
    
    folium.Popup(r['cluster_index']).add_to(geo_j)
    geo_j.add_to(m)
    
# cluster intersection map
gdf = cluster_on_route
for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: 
                           {'fillColor': 'red',
                            'color': 'red'}
    )
    
    folium.Popup(r['cluster_index']).add_to(geo_j)
    geo_j.add_to(m)
    
# add user itinerary polyline
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)

In [78]:
m

In [68]:
# export map
m.save("./templates/map.html")